<a href="https://colab.research.google.com/github/Riddick4-droid/Machine_Learning-Pt/blob/main/MovieRecommendation_Naivebayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##make imports
import pandas as pd
import numpy as np
import zipfile
import os

In [ ]:
##unzip
path = '/content/'
files_folder = path + '/dataset/'

if not os.path.exists(files_folder):
    os.makedirs(files_folder,exist_ok=True)

#unzip the file
with zipfile.ZipFile('/content/ml-1m.zip','r') as zipref:
    zipref.extractall(files_folder)

In [ ]:
##load the dataset
df = pd.read_csv('/content/dataset/ml-1m/ratings.dat',header=None,sep='::',engine='python',encoding='ISO-8859-1')

#view the dataset
df.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
##set columns
df.columns = ['user_id','movie_id','rating','timestamp']

In [ ]:
df.head(20)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [ ]:
df[df['user_id']==2]

,user_id,movie_id,rating,timestamp
53,2,1357,5,978298709
54,2,3068,4,978299000
55,2,1537,4,978299620
56,2,647,3,978299351
57,2,2194,4,978299297
...,...,...,...,...
177,2,356,5,978299686
178,2,1245,2,978299200
179,2,1246,5,978299418
180,2,3893,1,978299535


In [ ]:
##see unqiue movie ids
n_users = df['user_id'].nunique()
print(f'number of unique user ids: {n_users}')

##number of unique moviees
n_movies = df['movie_id'].nunique()
print(f'number of unique movies: {n_movies}')

#number of unique ratings
ratings = df['rating'].nunique()
print(f'number of unique ratings: {ratings}')

number of unique user ids: 6040
number of unique movies: 3706
number of unique ratings: 5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   movie_id   1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [ ]:
def load_user_rating_data(df, n_users, n_movies):
    """Creates a user-movie rating matrix and mappings"""
    data = np.zeros([n_users, n_movies], dtype=np.intc)
    movie_id_mapping = {}
    user_id_mapping = {}

    for user_id, movie_id, rating in zip(df['user_id'], df['movie_id'], df['rating']):
        # map the user_id to a row index
        if user_id not in user_id_mapping:
            user_id_mapping[user_id] = len(user_id_mapping)

        # mpa the movie_id to a column index
        if movie_id not in movie_id_mapping:
            movie_id_mapping[movie_id] = len(movie_id_mapping)

        # Assign rating
        data[user_id_mapping[user_id], movie_id_mapping[movie_id]] = rating

    return data, user_id_mapping, movie_id_mapping

In [ ]:
data,user_id_mapping,movie_id_mapping= load_user_rating_data(df,n_users,n_movies)

In [ ]:
pd.DataFrame(data,columns=[f'mov_{i}'for i in range(data.shape[1])])

,mov_0,mov_1,mov_2,mov_3,mov_4,mov_5,mov_6,mov_7,mov_8,mov_9,...,mov_3696,mov_3697,mov_3698,mov_3699,mov_3700,mov_3701,mov_3702,mov_3703,mov_3704,mov_3705
0,5,3,3,4,5,3,5,5,4,4,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,5,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,3,5,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,5,0,3,4,4,5,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
6036,4,0,0,0,0,0,0,4,0,4,...,0,0,0,0,0,0,0,0,0,0
6037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6038,0,3,4,0,0,4,0,4,0,4,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
movie_id_mapping

{1193: 0,
 661: 1,
 914: 2,
 3408: 3,
 2355: 4,
 1197: 5,
 1287: 6,
 2804: 7,
 594: 8,
 919: 9,
 595: 10,
 938: 11,
 2398: 12,
 2918: 13,
 1035: 14,
 2791: 15,
 2687: 16,
 2018: 17,
 3105: 18,
 2797: 19,
 2321: 20,
 720: 21,
 1270: 22,
 527: 23,
 2340: 24,
 48: 25,
 1097: 26,
 1721: 27,
 1545: 28,
 745: 29,
 2294: 30,
 3186: 31,
 1566: 32,
 588: 33,
 1907: 34,
 783: 35,
 1836: 36,
 1022: 37,
 2762: 38,
 150: 39,
 1: 40,
 1961: 41,
 1962: 42,
 2692: 43,
 260: 44,
 1028: 45,
 1029: 46,
 1207: 47,
 2028: 48,
 531: 49,
 3114: 50,
 608: 51,
 1246: 52,
 1357: 53,
 3068: 54,
 1537: 55,
 647: 56,
 2194: 57,
 648: 58,
 2268: 59,
 2628: 60,
 1103: 61,
 2916: 62,
 3468: 63,
 1210: 64,
 1792: 65,
 1687: 66,
 1213: 67,
 3578: 68,
 2881: 69,
 3030: 70,
 1217: 71,
 434: 72,
 2126: 73,
 3107: 74,
 3108: 75,
 3035: 76,
 1253: 77,
 1610: 78,
 292: 79,
 2236: 80,
 3071: 81,
 902: 82,
 368: 83,
 1259: 84,
 3147: 85,
 1544: 86,
 1293: 87,
 1188: 88,
 3255: 89,
 3256: 90,
 3257: 91,
 110: 92,
 2278: 93,
 24

In [ ]:
values,counts = np.unique(data,return_counts=True)

for value,count in zip(values,counts):
    print(f'number of rating {value}:{count}')

number of rating 0:21384031
number of rating 1:56174
number of rating 2:107557
number of rating 3:261197
number of rating 4:348971
number of rating 5:226310


In [ ]:
target_movie_id = 2858

X_raw = np.delete(data,movie_id_mapping[target_movie_id],axis=1)
y_raw = data[:,movie_id_mapping[target_movie_id]]

X = X_raw[y_raw > 0]
Y = y_raw[y_raw > 0]

print('shape of X:',X.shape)
print('shape of y: ',Y.shape)

shape of X: (3428, 3705)
shape of y:  (3428,)


In [ ]:
#setting up recommnndation threshold
recommend = 3
Y[Y<= recommend]=0 #0 means do not recommend
Y[Y> recommend]=1 #1 means recommend

#collkect count of recommended movies from the dataset
n_pos = (Y==1).sum()
#collect count of non-recommended movies from the dataset
n_neg = (Y==0).sum()

print(f'{n_pos} positive samples and {n_neg} negative samples.')

2853 positive samples and 575 negative samples.


There is a high class imbalance from the dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

In [ ]:
#get length of train and test data
len(X_train),len(X_test)

(2742, 686)

In [ ]:
#now to train the naive bayes algorithm
from sklearn.naive_bayes import MultinomialNB

#initialize
clf = MultinomialNB(alpha=1.0,fit_prior=True)

#train
clf.fit(X_train,y_train)

MultinomialNB()

In [ ]:
##predict on the test data
prediction_prob = clf.predict_proba(X_test)

#print
print(prediction_prob[:10])

[[7.50487439e-23 1.00000000e+00]
 [1.01806208e-01 8.98193792e-01]
 [3.57740570e-10 1.00000000e+00]
 [1.00000000e+00 2.94095407e-16]
 [1.00000000e+00 2.49760836e-25]
 [7.62630220e-01 2.37369780e-01]
 [3.47479627e-05 9.99965252e-01]
 [2.66075292e-11 1.00000000e+00]
 [5.88493563e-10 9.99999999e-01]
 [9.71326867e-09 9.99999990e-01]]


In [ ]:
##predict on the test data
prediction_prob = clf.predict(X_test)

#print
print(prediction_prob[:10])

[1 1 1 0 0 0 1 1 1 1]


In [ ]:
##get the accuracy score
accuracy = clf.score(X_test,y_test)

#print
print(f'the accuracy is: {accuracy*100:.1f}%')

the accuracy is: 71.6%


In [ ]:
##other model evaluation metrics

#confusion matrix
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test,prediction_prob,labels=[0,1]))

[[ 60  47]
 [148 431]]


In [ ]:
##precision, recall and f1 score metric

#precision->a fraction of positive calls that are correct
from sklearn.metrics import precision_score

precision = precision_score(y_test,prediction_prob)
print('precision=',precision)

#recall->measures the fraction of true positives that are measured correctly identified
from sklearn.metrics import recall_score

recall = recall_score(y_test,prediction_prob)
print('recall=',recall)

#f1score->includes both the precision and recall and serves as a middle ground
from sklearn.metrics import f1_score

f1score = f1_score(y_test,prediction_prob)
print('f1score=',f1score)

precision= 0.9016736401673641
recall= 0.7443868739205527
f1score= 0.815515610217597


In [ ]:
##all this can be achieved through the precision_recall_fscore_support
from sklearn.metrics import precision_recall_fscore_support

metrics = precision_recall_fscore_support(y_test,prediction_prob)

print('metrics=',metrics)

metrics= (array([0.28846154, 0.90167364]), array([0.56074766, 0.74438687]), array([0.38095238, 0.81551561]), array([107, 579]))


In [ ]:
##use classification report
from sklearn.metrics import classification_report

print(classification_report(y_test,prediction_prob,zero_division='warn'))

              precision    recall  f1-score   support

           0       0.29      0.56      0.38       107
           1       0.90      0.74      0.82       579

    accuracy                           0.72       686
   macro avg       0.60      0.65      0.60       686
weighted avg       0.81      0.72      0.75       686



The classification report provides indepth analysis of the model's performance

Precision: For each class (0 and 1), precision answers the question: "Of all the instances predicted as this class, how many actually belong to this class?" A high precision means fewer false positives.

For class 0, the precision is 0.29, meaning that when the model predicts class 0 (not recommended), it is correct 29% of the time.
For class 1, the precision is 0.90, meaning that when the model predicts class 1 (recommended), it is correct 90% of the time.
Recall: For each class, recall answers the question: "Of all the instances that actually belong to this class, how many did the model correctly identify?" A high recall means fewer false negatives.

For class 0, the recall is 0.56, meaning the model identifies 56% of the actual non-recommended movies.
For class 1, the recall is 0.74, meaning the model identifies 74% of the actual recommended movies.
F1-score: This is the harmonic mean of precision and recall. It provides a single score that balances both metrics. A higher F1-score indicates a better balance between precision and recall.

For class 0, the F1-score is 0.38.
For class 1, the F1-score is 0.82.
Support: This is the number of actual occurrences of each class in the y_test dataset.

There are 107 instances of class 0 (non-recommended).
There are 579 instances of class 1 (recommended).
Accuracy: This is the overall correctness of the model, calculated as the number of correct predictions divided by the total number of predictions. In your case, the overall accuracy is 0.72 (72%).

Macro avg: This calculates the average of precision, recall, and F1-score independently for each class and then takes the average. It treats all classes equally, regardless of their support.

Weighted avg: This calculates the average of precision, recall, and F1-score for each class, weighted by the number of true instances for each class (support). This is useful when you have imbalanced classes, as it accounts for the contribution of each class size to the overall average.

In [ ]:
##addressing the issue of overfitting and model tuning/selection
#model tuning or selection is the process of obtaining a set of hyperparameters for a model
#such that, that model returns on average a high performance
#this can be achieved throuigh cross validation

#implementing cross validation
from sklearn.model_selection import StratifiedKFold

#set k
k=5

#initialize kfold
kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [ ]:
##for the model tuning->we need to define a set of hyperparameters that we want to train the model on
#for each split in the cross-validation, the model will be initialized with a set of random combinations of these
#hyperparameters. for each split say split 1, we reserve 4 folds for training and 1 for validation.
#the model hyperparameters will be initialized in training the model and then the model will be used on the validation fold
##to validate the particular model with its initialized hyperparameters performance. This happens until all combinations of the hyperparameters
#are reached.
#each model combination undergoes k fold validations and the average validation score is taken.

#define the hyperparameters for tuning
smoothing_factor_option = [1,2,3,4,5,6]
fit_prior = [True,False]

#collect auc metric for each combination
auc_record = {}

In [ ]:
#implement kfold
from sklearn.metrics import roc_auc_score
for train_indices, test_indices in kfold.split(X,Y):
    #for each fold of the generated split() method of the k_fold
    #repeat the classifier initialization, training, and prediction
    #record the auc score per split
    x_train_k,x_test_k = X[train_indices],X[test_indices]
    y_train_k,y_test_k = Y[train_indices],Y[test_indices]

    for alpha in smoothing_factor_option:
        #esnure no duplicate in
        if alpha not in auc_record:
            #initialize record keeping for alpha-auc score
            auc_record[alpha]={}
            #iterate through the fit_prior hyperparameter
        for prior in fit_prior:
            clf = MultinomialNB(alpha=alpha,fit_prior=prior)
            #fit
            clf.fit(x_train_k,y_train_k)
            #predict
            prediction_prob = clf.predict_proba(x_test_k)
            #get positive prob column p(y=1)
            pos_prob = prediction_prob[:,1]
            #calculate the auc_score
            auc = roc_auc_score(y_test_k,pos_prob)
            #update the dictionary of results
            auc_record[alpha][prior]=auc + auc_record[alpha].get(prior,0.0)

In [ ]:
#presneting the results
for smoothing,smoothing_record in auc_record.items():
    for fit_prior, auc in smoothing_record.items():
        print(f'{smoothing} {fit_prior} {auc/k:.5f}')

1 True 0.65280
1 False 0.65274
2 True 0.65108
2 False 0.65274
3 True 0.65313
3 False 0.65322
4 True 0.65397
4 False 0.65430
5 True 0.65455
5 False 0.65448
6 True 0.65478
6 False 0.65404


In [ ]:
##selecting the best hyperparameter
#fit_prior=True and alpha=6

#initialize new model
clf = MultinomialNB(alpha=6,fit_prior=True)

#fit
clf.fit(X_train,y_train)

#collect the prediction probabilties
pred_prob = clf.predict_proba(X_test)

#get the p(y=1)
pos_col = pred_prob[:,1]

#calculate the auc score
score_auc = roc_auc_score(y_test,pos_col)

#print
print('auc score for best model:',round(score_auc,5))

auc score for best model: 0.68063
